In [1]:
import numpy as np
import torch

In [ ]:
class Perceptron:
    def __init__(self) -> None:
        self.activationfunction = None
        self.diff_activationfunction = None
        self.bias = None
        self.weights = None
    
    def set_activation_function(self, name):
        if name.lower() == 'relu':
            self.activation_function = lambda x: np.maximum(0, x)
            self.diff_activation_function = lambda x: np.where(x > 0, 1, 0)
        elif name.lower() == 'sigmoid':
            self.activation_function = lambda x: 1 / (1 + np.exp(-x))
            self.diff_activation_function = lambda x: self.activation_function(x) * (1 - self.activation_function(x))
        elif name.lower() == 'tanh':
            self.activation_function = lambda x: np.tanh(x)
            self.diff_activation_function = lambda x: 1 - np.tanh(x)**2
        else:
            raise ValueError("Unsupported activation function. Choose 'relu', 'sigmoid', or 'tanh'.")